****GOAL OF THE MODEL****

**The business goal is to estimate the probability that an outstanding exposure will be collected within a given horizon, conditional on its current age and features.**

**“Given how long an account has had a margin call open, what is the probability that it will finish paying (i.e., settle its outstanding obligation) within the next X days?”**

So our survival function 𝑆(𝑡) means:

- *“Probability that the margin call is still unpaid after t days.”*

and the cumulative hazard (or 1–S(t)) means:

- *“Probability that the margin call has been settled within t days.”*

In our model, an **event** means the margin call has been fully paid — the account’s outstanding balance has dropped to nearly zero.

The model estimates how long it typically takes for clients to clear their obligations and what factors influence faster or slower repayment.

Each margin call is tracked from its issue date until it is either collected (event_date) or reaches the model cutoff (censor_date).
The model learns how the probability of recovery changes with the age of the margin call (age_since_issue) and other risk factors.
Observed durations (time between issue and collection/cutoff) provide the foundation for estimating how likely a currently open margin call is to be recovered over the next 30, 150, or 300 days.

The model considers key factors such as account risk profile, issued amount, payment progress, and the age of each margin call. Generally, calls with lower risk ratings, smaller issued amounts, and recent issuance dates have higher probabilities of being collected within the next periods. As calls age or remain largely unpaid, their recovery probability tends to decline. These estimates help identify which pending calls are more or less likely to be recovered over time.

In [ ]:
import os
import pandas as pd
import numpy as np
from google.cloud import bigquery
from lifelines import CoxPHFitter, KaplanMeierFitter
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# --- Mode selection ---
today = pd.Timestamp.today().normalize()

# Generate the business calendar for the current month
month_days = pd.bdate_range(today.replace(day=1), today + pd.offsets.MonthEnd(0))
first_business_day = month_days[0]

if today == first_business_day:
    mode = "score"
else:
    mode = "train"

print(f"✅ Mode set to: {mode.upper()} | First business day: {first_business_day.date()}")

# --- BigQuery Configuration ---
PROJECT_ID = 'example_projectid_dev'
DATASET_ID = 'alfonso_dataset'
OUTPUT_TABLE = f'{PROJECT_ID}.{DATASET_ID}.model_predictions'
OUTPUT_TABLE_AB = f'{PROJECT_ID}.{DATASET_ID}.model_agebucket'

# --- File paths ---
BASE_DIR = r"C:\\Users\\alfonso_hidalgo\\Documents\\my_rep\\my_models\\alm\\margin_call"
SQL_TRAIN_PATH = "`train_dataset_sql`"
SQL_SCORE_PATH = "`score_dataset_sql"
MODEL_PATH = os.path.join(BASE_DIR, 'coxph_model.pkl')

# --- Prediction horizons ---
prediction_horizons = [30, 150, 300]  # in days

# --- Initialize BigQuery client ---
client = bigquery.Client(project=PROJECT_ID)

print(f"✅ Environment ready | MODE: {mode.upper()}")
print(f"📦 Output table: {OUTPUT_TABLE}")

# ==============================
# 2. LOAD SQL AND FETCH DATA
# ==============================

sql_path = SQL_TRAIN_PATH if mode == "train" else SQL_SCORE_PATH
print(f"[INFO] Loading SQL source for mode={mode}: {sql_path}")

# --- Detect if sql_path is a BigQuery table reference or a local file
if sql_path.strip().lower().endswith(".sql"):
    # CASE 1: Local SQL file (development mode)
    with open(sql_path, "r", encoding="utf-8") as file:
        sql_query = file.read()
    print(f"[INFO] Loaded SQL from local file: {sql_path}")

    df = client.query(sql_query).to_dataframe()
else:
    # CASE 2: BigQuery table reference (production mode)
    table_ref = sql_path.strip("`")  # remove backticks if any
    print(f"[INFO] Reading directly from BigQuery table: {table_ref}")
    df = client.query(f"SELECT * FROM `{table_ref}`").to_dataframe()
    # List of date columns to normalize
    date_cols = ["issued_date", "eod_book", "censor_date"]
    for col in date_cols:
        df[col] = pd.to_datetime(df[col], errors="coerce")

print(f"[INFO] Data fetched from BigQuery | Rows: {len(df):,d} | Columns: {df.shape[1]}")

In [ ]:
df.groupby("dataset_type")["internal_risk_profile_score"].describe()

In [ ]:
df_static = df[df["dataset_type"] == "static"].copy()
df_landmark = df[df["dataset_type"] == "landmark"].copy()

print("Static shape:", df_static.shape)
print("Landmark shape:", df_landmark.shape)

# Confirm event balance
print(df_static["is_event"].value_counts(normalize=True))
print(df_landmark["is_event"].value_counts(normalize=True))

In [ ]:
sns.ecdfplot(data=df, x="observed_duration", hue="event_type")
plt.title("Empirical CDF: Time to Event vs Censored")
plt.show()

In [ ]:
# ============================================================
# 3. DATA PREPARATION - USING NEW SQL OUTPUT (STATIC + LANDMARK)
# ============================================================
df_prep = df.copy()
print(f"[INFO] Loaded dataset with {len(df_prep)} rows and {len(df_prep.columns)} columns")

# ------------------------------------------------------------
# 3.1. Basic Sanity Filters
# ------------------------------------------------------------
# Drop rows with missing duration or event info
df_prep = df_prep.dropna(subset=["observed_duration", "is_event"]).copy()

# Remove negative or zero durations
df_prep = df_prep[df_prep["observed_duration"] > 0].copy()

# ------------------------------------------------------------
# 3.2. Handle Missing / Noisy Numeric Fields
# ------------------------------------------------------------
fill_zero_cols = ["amount_issued", "amount_collected", "last_outstanding"]
for col in fill_zero_cols:
    if col in df_prep.columns:
        df_prep[col] = df_prep[col].fillna(0)

# ------------------------------------------------------------
# 3.3. Impute Missing Risk Scores and Normalize
# ------------------------------------------------------------
if "internal_risk_profile_score" in df_prep.columns:
    median_risk = df_prep["internal_risk_profile_score"].median()
    df_prep["internal_risk_profile_score"] = df_prep["internal_risk_profile_score"].fillna(median_risk)
    df_prep["risk_weight"] = (df_prep["internal_risk_profile_score"].clip(0, 100)) / 100
else:
    df_prep["risk_weight"] = 0.5

# ------------------------------------------------------------
# 3.4. Create Unified Modeling Variables
# ------------------------------------------------------------
df_prep["event_observed"] = df_prep["is_event"].astype(int)
df_prep["time_to_event"] = df_prep["observed_duration"].astype(float)

# Optional: Filter static or landmark subsets
df_static = df_prep[df_prep["dataset_type"] == "static"].copy()
df_landmark = df_prep[df_prep["dataset_type"] == "landmark"].copy()

print(f"[INFO] Static subset: {len(df_static):,} rows | Landmark subset: {len(df_landmark):,} rows")

# Quick quality check
print(df_prep[["dataset_type", "is_event", "observed_duration"]].groupby("dataset_type").describe())


In [ ]:
# ==================================================
# 🔹 ADD NEW COVARIATES FOR REPAYMENT MODEL
# ==================================================

# Progress ratio: how much of the call has been repaid
df_prep["progress_ratio"] = (
    df_prep["amount_collected"] / df_prep["amount_issued"].replace(0, np.nan)
).clip(0, 1).fillna(0)

# Remaining ratio: fraction still unpaid
df_prep["remaining_ratio"] = (
    df_prep["last_outstanding"] / df_prep["amount_issued"].replace(0, np.nan)
).clip(0, 1).fillna(0)

# Age since issue at the time of snapshot
if "age_since_issue" in df_prep.columns:
    df_prep["age_since_issue"] = df_prep["age_since_issue"].clip(lower=0)
else:
    df_prep["age_since_issue"] = (
        (df_prep["eod_book"] - df_prep["issued_date"]).dt.days
    ).clip(lower=0)

# Optional: interaction term (risk × amount)
df_prep["risk_x_amount"] = df_prep["risk_weight"] * df_prep["amount_issued"]

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(10, 5))
sns.histplot(
    data=df, x="observed_duration",
    hue="dataset_type", bins=50, kde=True, multiple="stack"
)
plt.title("Distribution of Observed Durations: Static vs Landmark")
plt.show()

sns.countplot(
    data=df, x="event_type", hue="dataset_type"
)
plt.title("Event vs Censored Counts by Dataset Type")
plt.show()


- *The static model looks at each account’s entire history, from when it was issued to when it was either closed, defaulted, or reached the observation cut-off date.
It's like looking at someone’s whole financial life in one go.*

- *The landmark model, on the other hand, looks at each account repeatedly through time, creating “snapshots” (for example, every few months) and predicting what might happen within the next year. This lets us track risk in real time instead of waiting until the end of the loan’s life.*

In [ ]:
# ================================================================
# 4. MODEL TRAINING / LOADING (STATIC + LANDMARK)
# ================================================================
# --- File paths for saving both models ---
static_model_path = os.path.join(BASE_DIR, "coxph_model_static.pkl")
landmark_model_path = os.path.join(BASE_DIR, "coxph_model_landmark.pkl")
# --- Define shared CoxPH covariates ---
features = ["risk_weight", "amount_band", "progress_ratio", "remaining_ratio", "age_since_issue", "risk_x_amount"]
# Helper: Fit and Save a Cox Model
def fit_cox_model(df, model_path, label):
    print(f"\n🧩 Training {label.upper()} Cox Proportional Hazards model...")

    # --- Define labels ---
    duration_col = "observed_duration"
    event_col = "is_event"

    # --- Encode categorical variables ---
    df_enc = pd.get_dummies(df[features], columns=["amount_band"], drop_first=False)

    # --- Combine with labels ---
    df_fit = df_enc.join(df[[duration_col, event_col]])
    print("[DEBUG] Columns with NaNs:")
    print(df_fit.isna().sum()[df_fit.isna().sum() > 0])

    # --- Clean missing or invalid values ---
    n_before = len(df_fit)
    df_fit = df_fit.replace([np.inf, -np.inf], np.nan).dropna()
    n_after = len(df_fit)
    print(f"[CLEANUP] Removed {n_before - n_after} rows with NaN or infinite values")

    # --- Fit CoxPH model ---
    cph = CoxPHFitter(penalizer=0.01)
    cph.fit(df_fit, duration_col=duration_col, event_col=event_col, show_progress=True)

    # --- Save model ---
    joblib.dump(cph, model_path)
    print(f"✅ {label.capitalize()} model trained and saved to {model_path}")
    print(f"Concordance Index ({label}): {cph.concordance_index_:.3f}\n")
    return cph

# Automatically train or load both Cox models
if mode == "train":
    # --- Prepare static dataset ---
    df_static_train = df_prep[df_prep["dataset_type"] == "static"].copy()
    df_static_train["time_to_event"] = df_static_train["observed_duration"].astype(float)
    df_static_train["event_observed"] = df_static_train["is_event"].astype(int)
    # --- Prepare landmark dataset ---
    df_landmark_train = df_prep[df_prep["dataset_type"] == "landmark"].copy()
    df_landmark_train["time_to_event"] = df_landmark_train["observed_duration"].astype(float)
    df_landmark_train["event_observed"] = df_landmark_train["is_event"].astype(int)
    # Sanity check
    print(f"\n[INFO] Static rows: {len(df_static_train):,}, Landmark rows: {len(df_landmark_train):,}")
    print(f"Static durations valid? {(df_static_train['time_to_event'] > 0).mean():.2%}")
    print(f"Landmark durations valid? {(df_landmark_train['time_to_event'] > 0).mean():.2%}")
    # --- Fit both models ---
    cph_static = fit_cox_model(df_static_train, static_model_path, "static")
    cph_landmark = fit_cox_model(df_landmark_train, landmark_model_path, "landmark")
else:
    # --- Load pre-trained models for scoring ---
    print("\n📂 Loading pre-trained CoxPH models...")
    cph_static = joblib.load(static_model_path)
    cph_landmark = joblib.load(landmark_model_path)
    print(f"✅ Models loaded successfully from {BASE_DIR}")

In [ ]:
# ✅ Ensure your main dataframe is loaded as 'df'
# (with columns like dataset_type, is_event, observed_duration, risk_weight, etc.)

sns.set(style="whitegrid", palette="muted")

# 1️⃣ — Basic overview
print(df.groupby("dataset_type")[["observed_duration", "internal_risk_profile_score"]].describe())

# 2️⃣ — Distribution comparison for key covariates
key_covariates = ["observed_duration", "internal_risk_profile_score"]

for col in key_covariates:
    if col not in df.columns:
        print(f"[WARN] Skipping missing column: {col}")
        continue
    plt.figure(figsize=(8, 4))
    sns.kdeplot(data=df, x=col, hue="dataset_type", fill=True, common_norm=False, alpha=0.4)
    plt.title(f"Distribution of {col}: Static vs Landmark")
    plt.show()

# 3️⃣ — Pairplot to visually check relationships
sample_df = df.sample(frac=0.05, random_state=42)  # sample for speed
sns.pairplot(sample_df,
             vars=["observed_duration", "internal_risk_profile_score"],
             hue="dataset_type", diag_kind="kde", corner=True)
plt.suptitle("Pairwise Feature Relationships — Static vs Landmark", y=1.02)
plt.show()

# 4️⃣ — Correlation comparison
for ds in ["static", "landmark"]:
    corr = df[df["dataset_type"] == ds][
        ["observed_duration", "internal_risk_profile_score"]
    ].corr()
    plt.figure(figsize=(4, 3))
    sns.heatmap(corr, annot=True, cmap="coolwarm", center=0, fmt=".2f")
    plt.title(f"Correlation Matrix ({ds.capitalize()})")
    plt.show()


In [ ]:
# ------------------------------------------------
# Optional: Quick visual diagnostics
# ------------------------------------------------
if mode == "train":
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    cph_static.plot(ax=axes[0])
    axes[0].set_title("Static Model: log(HR) ±95% CI")

    cph_landmark.plot(ax=axes[1])
    axes[1].set_title("Landmark Model: log(HR) ±95% CI")
    plt.tight_layout()
    plt.show()

    # Plot partial effects for comparison
    cph_static.plot_partial_effects_on_outcome(
        covariates="risk_weight",
        values=[0.25, 0.5, 0.75, 1.0],
        cmap="Blues",
        ax=None,
    )
    plt.title("Static Model — Partial Effect of Risk Weight")
    plt.show()

    cph_landmark.plot_partial_effects_on_outcome(
        covariates="risk_weight",
        values=[0.25, 0.5, 0.75, 1.0],
        cmap="Oranges",
        ax=None,
    )
    plt.title("Landmark Model — Partial Effect of Risk Weight")
    plt.show()


In [ ]:
# ===============================================================
# 5. MODEL SCORING / PREDICTION (STATIC + LANDMARK)
# ===============================================================

# --- Define prediction horizons (in days) ---
prediction_horizons = [30, 150, 300]  # adjust these as needed

# --- Choose subsets for scoring ---
df_static_score = df_prep[df_prep["dataset_type"] == "static"].copy()
df_landmark_score = df_prep[df_prep["dataset_type"] == "landmark"].copy()

# --- Ensure feature formatting is correct ---
def prepare_features(df, cph_model):
    df_sc = df.copy()

    df_sc["time_to_event"] = df_sc["observed_duration"].astype(float)
    df_sc["event_observed"] = df_sc["is_event"].astype(int)
    df_sc["risk_weight"] = df_sc["risk_weight"].astype(float)

    # One-hot encode amount_band
    df_sc = pd.get_dummies(df_sc, columns=["amount_band"], drop_first=False)

    # Align columns with model
    expected_cols = cph_model.params_.index
    for col in expected_cols:
        if col not in df_sc.columns:
            df_sc[col] = 0
    df_sc = df_sc[expected_cols]

    return df_sc

df_static_enc = prepare_features(df_static_score, cph_static)
df_landmark_enc = prepare_features(df_landmark_score, cph_landmark)

# ---------------------------------------------------------------
# Helper: generate survival predictions for given horizons
# ---------------------------------------------------------------
def generate_survival_predictions(cph_model, df_enc, horizons, label):
    print(f"\n🧮 Generating survival predictions for {label.upper()} dataset...")

    results = []

    # Predict only for selected horizons, not full curve
    for h in horizons:
        surv_probs = cph_model.predict_survival_function(df_enc, times=[h])
        df_temp = pd.DataFrame({
            "reference_id": df_enc.index,
            f"surv_{h}d": surv_probs.T.iloc[:, 0].values,
            "dataset_type": label
        })

        results.append(df_temp)

    # Merge all horizon predictions
    surv_results = results[0]
    for i in range(1, len(results)):
        surv_results = surv_results.merge(results[i], on=["reference_id", "dataset_type"], how="left")

    return surv_results

# ---------------------------------------------------------------
# Generate predictions for both models
# ---------------------------------------------------------------
pred_static = generate_survival_predictions(cph_static, df_static_enc, prediction_horizons, "static")
pred_landmark = generate_survival_predictions(cph_landmark, df_landmark_enc, prediction_horizons, "landmark")

# Merge them together
predictions_all = pd.concat([pred_static, pred_landmark], ignore_index=True)

print("\n✅ Predictions generated successfully!")
print(predictions_all.head())


In [ ]:
sns.histplot(predictions_all, x="surv_300d", hue="dataset_type", bins=50, kde=True)


In [ ]:
# ============================================================
# 🧠 MODEL SCORING PIPELINE — PRODUCTION (LANDMARK MODEL)
# ============================================================

client = bigquery.Client(project=PROJECT_ID)

# --- Prediction horizons ---
prediction_horizons = [30, 150, 300]  # in days

# --- Load pre-trained landmark model ---
landmark_model_path = os.path.join(BASE_DIR, "coxph_model_landmark.pkl")
cph_landmark = joblib.load(landmark_model_path)
print("✅ Landmark model loaded successfully.")

# ============================================================
# 1️⃣ LOAD CURRENT SNAPSHOT FROM BIGQUERY
# ============================================================

table_ref = SQL_SCORE_PATH.strip("`")
print(f"[INFO] Reading live snapshot from BigQuery table: {table_ref}")

df_score_raw = client.query(f"SELECT * FROM `{table_ref}`").to_dataframe()
print(f"[INFO] Loaded {len(df_score_raw):,} live records for scoring.")

# ============================================================
# 🧩 FEATURE ENGINEERING for score_dataset
# ============================================================

df_score_raw = client.query(f"SELECT * FROM `{table_ref}`").to_dataframe()
print(f"[INFO] Loaded {len(df_score_raw):,} live records for scoring.")

# --- 1. Normalize risk_weight (0–1 scale) ---
df_score_raw["risk_weight"] = (
    df_score_raw["internal_risk_profile_score"].fillna(50).clip(0, 100) / 100
)

# --- 2. Derive amount_band (categorical) ---
df_score_raw["amount_band"] = pd.cut(
    df_score_raw["gbp_issued"],
    bins=[-float("inf"), 5000, 20000, float("inf")],
    labels=["Small", "Medium", "Large"]
)

# --- 3. Compute progress_ratio (how much collected) ---
df_score_raw["progress_ratio"] = (
    df_score_raw["gbp_collected"] / df_score_raw["gbp_issued"]
).replace([np.inf, -np.inf], np.nan).fillna(0)

# --- 4. Compute remaining_ratio (how much still to collect) ---
df_score_raw["remaining_ratio"] = 1 - df_score_raw["progress_ratio"]

# --- 5. Assign age_since_issue ---
df_score_raw["age_since_issue"] = df_score_raw["days_from_issued_date"].fillna(0)

# --- 6. Risk × amount interaction ---
df_score_raw["risk_x_amount"] = df_score_raw["risk_weight"] * df_score_raw["gbp_issued"]

print("[INFO] Derived all model features successfully.")

# ============================================================
# 2️⃣ PREPARE FEATURES
# ============================================================

def prepare_features(df, cph_model):
    df_sc = df.copy()
    # --- Keep ID and descriptive columns ---
    keep_cols = ["reference_id", "account_name"]
    # Ensure feature types
    df_sc["risk_weight"] = df_sc["risk_weight"].astype(float)
    df_sc["age_since_issue"] = df_sc["age_since_issue"].astype(float)
    
    # One-hot encode categorical variables
    df_sc = pd.get_dummies(df_sc, columns=["amount_band"], drop_first=False)
    
    # Align with model expected columns
    expected_cols = cph_model.params_.index
    for col in expected_cols:
        if col not in df_sc.columns:
            df_sc[col] = 0
    # --- Final order: IDs + model features ---
    df_sc = df_sc[keep_cols + list(expected_cols)]
    return df_sc

df_score_enc = prepare_features(df_score_raw, cph_landmark)
print("[INFO] Features prepared for production scoring.")

# ============================================================
# 3️⃣ GENERATE SURVIVAL + COLLECTION PROBABILITIES
# ============================================================

def generate_survival_predictions(cph_model, df_enc, horizons, label):
    results = []
    for h in horizons:
        surv_probs = cph_model.predict_survival_function(df_enc.drop(columns=["reference_id", "account_name"]), times=[h])
        df_temp = pd.DataFrame({
            "reference_id": df_enc["reference_id"].values,
            "account_name": df_enc["account_name"].values,
            f"surv_{h}d": surv_probs.T.iloc[:, 0].values,
            "dataset_type": label
        })
        results.append(df_temp)

    surv_results = results[0]
    for i in range(1, len(results)):
        surv_results = surv_results.merge(results[i], on=["reference_id", "account_name", "dataset_type"], how="left")
    return surv_results

# --- Generate survival predictions ---
pred_prod = generate_survival_predictions(cph_landmark, df_score_enc, prediction_horizons, "production")

# --- Compute collection probabilities ---
for h in prediction_horizons:
    pred_prod[f"prob_collected_{h}d"] = 1 - pred_prod[f"surv_{h}d"]

# Add timestamp
pred_prod["scoring_date"] = datetime.now().date()

print("✅ Predictions generated successfully.")
print(pred_prod.head())

print(pred_prod[["reference_id", "account_name"]].head())
print(pred_prod.dtypes)

# ============================================================
# 4️⃣ EXPORT RESULTS BACK TO BIGQUERY
# ============================================================

job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
client.load_table_from_dataframe(pred_prod, OUTPUT_TABLE, job_config=job_config).result()
print(f"✅ Predictions uploaded to BigQuery table: {OUTPUT_TABLE}")

In [ ]:
sns.histplot(df_score_raw, x="progress_ratio", bins=50, kde=True)
plt.title("Progress Ratio Distribution (Live Margin Calls)")
plt.show()

- Near 0.0 = Outstanding / not collected = Active margin calls still awaiting payment.

- Near 1.0 = Fully collected = Margin calls that have already been settled.

- Middle (0.2–0.8) = Partial collections = Operationally rare, clients tend to pay all or none.

In [ ]:
sns.scatterplot(data=df_score_raw, x="age_since_issue", y="progress_ratio", alpha=0.5)
plt.title("Progress vs. Age Since Issue (Live Margin Calls)")
plt.xlabel("Days since issued")
plt.ylabel("Progress ratio (Collected / Issued)")
plt.show()

Most points are clustered vertically around very low ages (0–100 days).

Within that cluster, progress ratios are mostly near 0 (unpaid) and 1 (fully paid).

A few calls extend beyond 500–1000 days, these are long-standing MCs, probably residuals or historical outliers that never fully cleared.

✅ This confirms the pattern saw in the previous histogram: Most live margin calls are either brand new (unpaid) or fully collected.

In [ ]:
# ============================================
# 🧮 AGE BUCKET ANALYSIS – Weighted Probabilities + Expected Recovery + Total Row
# ============================================

# --- Define 30-day buckets up to 1 year, then "Over 1 year" ---
bins = list(range(0, 361, 30)) + [float("inf")]
labels = [f"{i}-{i+29}d" for i in range(0, 360, 30)] + ["Over 1 year"]

df_age = df_score_raw.copy()
df_age["age_bucket"] = pd.cut(df_age["age_since_issue"], bins=bins, labels=labels, right=False)

# --- Assign alphabetical order for sorting ---
bucket_order_map = {
    "0-29d": "a", "30-59d": "b", "60-89d": "c", "90-119d": "d",
    "120-149d": "e", "150-179d": "f", "180-209d": "g", "210-239d": "h",
    "240-269d": "i", "270-299d": "j", "300-359d": "k", "Over 1 year": "l"
}
df_age["age_bucket_order"] = df_age["age_bucket"].map(bucket_order_map)

# --- Merge model probabilities ---
df_age = df_age.merge(
    pred_prod[["reference_id", "prob_collected_30d", "prob_collected_150d", "prob_collected_300d"]],
    on="reference_id", how="left"
)

# --- Compute outstanding exposure globally ---
df_age["outstanding"] = df_age["gbp_issued"] - df_age["gbp_collected"]

# --- Compute metrics by age bucket ---
summary = (
    df_age.groupby(["age_bucket", "age_bucket_order"], observed=True, group_keys=False)
    .apply(lambda g: pd.Series({
        "n_accounts": len(g),

        # Average probabilities (unweighted)
        "avg_prob_30d": g["prob_collected_30d"].mean(),
        "avg_prob_150d": g["prob_collected_150d"].mean(),
        "avg_prob_300d": g["prob_collected_300d"].mean(),

        # Weighted probabilities (using outstanding as weight)
        "wgt_prob_30d": (g["prob_collected_30d"] * g["outstanding"]).sum() / g["outstanding"].sum() if g["outstanding"].sum() > 0 else np.nan,
        "wgt_prob_150d": (g["prob_collected_150d"] * g["outstanding"]).sum() / g["outstanding"].sum() if g["outstanding"].sum() > 0 else np.nan,
        "wgt_prob_300d": (g["prob_collected_300d"] * g["outstanding"]).sum() / g["outstanding"].sum() if g["outstanding"].sum() > 0 else np.nan,

        # Expected recoveries (in £)
        "expected_recovery_30d": (g["outstanding"] * g["prob_collected_30d"]).sum(),
        "expected_recovery_150d": (g["outstanding"] * g["prob_collected_150d"]).sum(),
        "expected_recovery_300d": (g["outstanding"] * g["prob_collected_300d"]).sum(),

        # Exposure metrics
        "avg_gbp_issued": g["gbp_issued"].mean(),
        "total_gbp_issued": g["gbp_issued"].sum(),
        "total_gbp_collected": g["gbp_collected"].sum(),
        "total_gbp_outstanding": g["outstanding"].sum(),
    }))
    .reset_index()
)

# --- Add traceability metadata ---
summary["scoring_date"] = pd.Timestamp.today().normalize()
summary["scoring_timestamp"] = datetime.now()
summary["source_table"] = "int_4_cl_mc_score"
summary["model_table"] = "int_5_cl_mc_predictions"
summary["model_version"] = "landmark_v1.0"

# --- Ensure BigQuery-compatible types ---
for col in ["age_bucket", "age_bucket_order"]:
    summary[col] = summary[col].astype(str)

# --- Sort and export ---
summary = summary.sort_values("age_bucket_order").reset_index(drop=True)

job_config = bigquery.LoadJobConfig(write_disposition="WRITE_TRUNCATE")
client.load_table_from_dataframe(summary, OUTPUT_TABLE_AB, job_config=job_config).result()

print(f"\n✅ Age bucket summary uploaded successfully to {OUTPUT_TABLE_AB}")
print(f"Rows uploaded: {len(summary)}, Columns: {len(summary.columns)}")


In [ ]:
# Define the order of buckets explicitly
bucket_order = [
    "0-29d", "30-59d", "60-89d", "90-119d",
    "120-149d", "150-179d", "180-209d", "210-239d",
    "240-269d", "270-299d", "300-359d", "Over 1 year"
]

# Convert to ordered categorical
summary["age_bucket"] = pd.Categorical(
    summary["age_bucket"],
    categories=bucket_order,
    ordered=True
)

# Sort by that order
summary = summary.sort_values("age_bucket")

plt.figure(figsize=(10, 6))
sns.lineplot(data=summary, x="age_bucket", y="wgt_prob_30d", label="Wgt Prob 30d")
sns.lineplot(data=summary, x="age_bucket", y="wgt_prob_150d", label="Wgt Prob 150d")
sns.lineplot(data=summary, x="age_bucket", y="wgt_prob_300d", label="Wgt Prob 300d")

plt.title("Weighted Collection Probability by Age Bucket (Live Margin Calls)")
plt.xlabel("Age since issue (buckets)")
plt.ylabel("Probability of being collected")
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

**This chart adjusts for the size of each exposure, so it shows the overall portfolio’s expected cash recovery, not just the count of calls.**

- Larger margin calls tend to be paid slightly faster, which improves early-bucket performance.

- But once a large call remains unpaid for several months, the likelihood of collecting it drops even more sharply.

- This means the biggest risks are long-aged, high-value calls, they carry most of the financial exposure but very low recovery potential.

****Summary****

- Avg. Coll. Prob. = The longer a margin call stays unpaid, the smaller its chance of ever being recovered, fresh calls matter most.

- Wei. Coll. Prob. = Our biggest exposures behave the same way, quick payers clear fast, but old, large calls rarely recover.